In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

In [ ]:
import numpy as np

### Initial Deployment

In [ ]:
name = 'sphere_cap_0.3'

In [ ]:
input_path = '../../data/{}.json'.format(name)
target_mesh_path = '../../data/target_meshes/{}.obj'.format(name)

In [ ]:
# input_path = '../../data/{}.json'.format('hemisphere')
# target_mesh_path = '../../data/target_meshes/{}.obj'.format('hemisphere')

In [ ]:
from helpers.deployment_helper import get_deployed_umbrella_mesh

In [ ]:
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(target_mesh_path, io)
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))

In [ ]:
curr_um.energy(UmbrellaEnergyType.Full)

#### Pin Rigid Motion



In [ ]:
use_pin = False

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

In [ ]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

In [ ]:
import mesh
target_mesh = mesh.Mesh(target_mesh_path)
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
# view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# view.update(scalarField = rod_colors)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
angles = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])
    if (i % 10 == 0):
        view.update()
        view.showScalarField(rod_colors)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 0.1
# curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.Constant
curr_um.targetDeploymentHeight = thickness * 5
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 2000
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)

In [ ]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyRepulsion(), curr_um.energyAttraction()

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [ ]:
angles = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])
    if (i % 10 == 0):
        view.update()
        view.showScalarField(rod_colors)

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
curr_um.numDoF()

In [ ]:
results.success

### Initialize Design Optimization

In [ ]:
import umbrella_optimization
import umbrella_optimization_finite_diff

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, OPTS, 2.5, -1, False, fixedVars)

In [ ]:
optimizer.beta = 5 * 10e5

In [ ]:
optimizer.gamma

In [ ]:
optimizer.setHoldClosestPointsFixed(False, False)

In [ ]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Umbrella)

In [ ]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Target)

In [ ]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Full)

In [ ]:
original_design_parameters = curr_um.getDesignParameters()

In [ ]:
rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
single_rest_height_optimizer = umbrella_optimization.UmbrellaSingleRestHeightOptimization(rest_height_optimizer)

In [ ]:
class UmbrellaOptimizationCallback:
    def __init__(self, optimizer, umbrella, view, update_color = False, no_surface = False, callback_freq = 1):
        self.optimizer     = optimizer
        self.umbrella       = umbrella
        self.view  = view
        self.update_color = update_color
        self.no_surface    = no_surface
        self.callback_freq = callback_freq
        self.iterateData = []

    def __call__(self):
        if self.view and (len(self.iterateData) % self.callback_freq == 0):
            if self.update_color:
                self.view.showScalarField(rod_colors)
            else:
                pass
                self.view.update()
        return

    def numIterations(self): return len(self.iterateData)

In [ ]:
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
cb = UmbrellaOptimizationCallback(single_rest_height_optimizer, curr_um, view, True, False, 1)

In [ ]:
solverStatus = umbrella_optimization.optimize(single_rest_height_optimizer, algorithm, 20, 1.0, 1e-2, cb, -1)

In [ ]:
solverStatus

In [ ]:
curr_um.getDesignParameters()

In [ ]:
rest_height_optimizer.params()

In [ ]:
import numpy.linalg as la